___
# Import Library
___

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import category_encoders as ce

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.metrics import mean_squared_error

from dotenv import load_dotenv, find_dotenv, set_key
import sqlalchemy as db
import os
import pickle

In [2]:
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [3]:
load_dotenv()

True

In [4]:
RANDOM_STATE = 202102
SCORING = 'neg_root_mean_squared_error'

___
# Business Problems
___

Who doesn't like to travel? Traveling is a such fun activity, coming into new place and experience many new things. It is a part of crucial aspect in life as it's the best way to escape from hectic and busy schedule. It also improves the mental and physical health while it's a good remedy for stress, anxiety and depression. As part of travelling, deciding accomodation or where you should sleep and charge your body is definitely important thing to decide. One of the choice to spend your night, apart from conventional hotel is Airbnb.

Airbnb is an American vacation rental online marketplace company based in San Francisco, California. Airbnb maintains and hosts a marketplace, accessible to consumers on its website or via an app. Through the service, users can arrange lodging, primarily homestays, and tourism experiences or list their properties for rental. Airbnb does not own any of the listed properties; instead, it profits by receiving commission from each booking. 

Singapore, one of developed country in Asia, make 4.1% of its national GDP from tourism industry alone and has been one of the most visited country in Asia resulting in thousand of Airbnb listings in Singapore (around 4000+ listings). It can be troublesome to choose one befitting your needs and budget. This also becomes problem for owner property who wants to register their new property in Airbnb since they may find it hard to price their property.

Here's where machine learning help to solve those problems. Predicting price of property, or in this case Airbnb listing, could be quite challenging since there are various factors need to be measured and calculated in order to get price accurately. In this project, we will do end-to-end machine learning project, starting from cleaning the dataset, do exploratory data analysis to get some insights, use machine learning model to predict Airbnb listing price and deploy our best model by creating dashboard.   

___
# Load Dataset
___

In [5]:
engine = db.create_engine(os.getenv('db-uri'))

In [6]:
meta = db.MetaData()
meta.reflect(engine)

In [7]:
with engine.connect() as con:
    query = db.select([meta.tables['listings']])
    result = con.execute(query).fetchall()
df_raw = pd.DataFrame(result, columns=meta.tables['listings'].c.keys())

In [8]:
listings_dropped_columns = os.getenv('listings_dropped_columns')
listings_dropped_columns

"['id', 'listing_url', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month', 'host_response_rate', 'host_acceptance_rate', 'host_listings_count', 'host_total_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'availability_60', 'availability_90', 'availability_365', 'host_i

In [9]:
df = df_raw.drop(eval(listings_dropped_columns), axis=1)
display(df)

neighbourhood_cleansed neighbourhood_group_cleansed  \
0                 Woodlands                 North Region   
1               Bukit Timah               Central Region   
2                 Woodlands                 North Region   
3                  Tampines                  East Region   
4                  Tampines                  East Region   
...                     ...                          ...   
4382            Bukit Timah               Central Region   
4383          Marine Parade               Central Region   
4384                Geylang               Central Region   
4385          Marine Parade               Central Region   
4386                Kallang               Central Region   

                    property_type        room_type  accommodates  bedrooms  \
0       Private room in apartment     Private room             1       1.0   
1       Private room in apartment     Private room             2       1.0   
2       Private room in apartment     Private room             1       1.0   
3           Private room in villa     Private room             6       2.0   
4           Private room in house     Private room             3       1.0   
...                           ...              ...           ...       ...   
4382  Private room in condominium     Private room             2       NaN   
4383        Private room in house     Private room             1       1.0   
4384        Private room in house     Private room             4       1.0   
4385        Private room in house     Private room             1       1.0   
4386             Entire apartment  Entire home/apt             2       NaN   

      beds  price  minimum_nights  maximum_nights  availability_30  \
0      1.0     79             180             360               30   
1      1.0     80              90             730               30   
2      1.0     66               6              14               30   
3      3.0    174              90            1125               30   
4      1.0     93              90            1125               30   
...    ...    ...             ...             ...              ...   
4382   1.0     25              10              30                0   
4383   1.0     57              90             182               29   
4384   1.0     62              90             182               29   
4385   1.0     47              90             182               29   
4386   1.0    160               6            1125               29   

      instant_bookable  calculated_host_listings_count  total_bathrooms  \
0                False                               2              1.0   
1                False                               1              1.0   
2                False                               2              1.0   
3                False                               8              1.0   
4                False                               8              1.0   
...                ...                             ...              ...   
4382              True                               2              1.0   
4383              True                               6              1.0   
4384              True                               6              1.0   
4385              True                               6              1.0   
4386             False                             126              1.0   

        bathrooms_type  Air Conditioning  BBQ Utensils  \
0                 bath                 1             0   
1                 bath                 1             0   
2                 bath                 1             0   
3         private bath                 1             0   
4     shared half-bath                 1             0   
...                ...               ...           ...   
4382       shared bath                 1             0   
4383         half-bath                 1             0   
4384              bath                 1             0   
4385         half-bath   

___
# Data Splitting
___

In [10]:
X = df.drop('price', axis=1).fillna(np.nan)
y = df['price']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=RANDOM_STATE)

In [12]:
for x in [X_train, X_test, y_train, y_test]:
    print(x.shape)

(3509, 64)
(878, 64)
(3509,)
(878,)


In [13]:
list_column_input = X_train.columns.tolist()
if os.getenv('list_column_input') != list_column_input:
    set_key(find_dotenv(), 'list_column_input', str(list_column_input))

___
# Choose Evaluation Metric
___

In this project, one of our goal is to predict Airbnb listing as accurately as possible (regression problem). Hence, I choose RMSE (Root Mean Squared Error) as evaluation metric since it shows how far our set of predictions to the actual price and its ability to punish high difference of prediction and actual price.

___
# Data Transformer
___

In [14]:
encode_df = pd.DataFrame()
encode_df['Unique Values'] = df.apply(pd.unique)
encode_df['Count'] = encode_df['Unique Values'].apply(len)
encode_df['Data Type'] = [df[t].dtype for t in encode_df.index]
encode_df['%Missing'] = df.isnull().sum()/len(df)*100
encode_df.sort_values('Count')

Unique Values  \
Entertainment                                                                    [0, 1]   
Ethernet Connection                                                              [1, 0]   
Extra pillows, blankets or bed-linen                                             [0, 1]   
Fan                                                                              [0, 1]   
Fire Safety Equipments                                                           [0, 1]   
Fireplace                                                                        [0, 1]   
First Aid Kit                                                                    [0, 1]   
Free Parking                                                                     [0, 1]   
Gym                                                                              [0, 1]   
Heating                                                                          [0, 1]   
Host Greets You                                                                  [0, 1]   
Hot Tub                                                                          [0, 1]   
Hot Water                                                                        [0, 1]   
Kitchen Utensils                                                                 [0, 1]   
Kitchen/Dining Area                                                              [0, 1]   
Laundromat Nearby                                                                [0, 1]   
Living Room                                                                      [0, 1]   
Lockbox                                                                          [0, 1]   
Long Term Stays Allowed                                                          [0, 1]   
Luggage Dropoff Allowed                                                          [0, 1]   
Outdoor Space                                                                    [0, 1]   
Paid Parking                                                                     [0, 1]   
Pool                                                                             [0, 1]   
Private Entrance                                                                 [0, 1]   
Refrigerator/Freezer                                                             [0, 1]   
Sauna                                                                            [0, 1]   
Single level Home                                                                [0, 1]   
Ski-in/Ski-out                                                                   [0, 1]   
TV                                                                               [1, 0]   
Toilet Equipments                                                                [0, 1]   
Essentials                                                                       [0, 1]   
Washer                                                                           [1, 0]   
Water Body Access                                                                [0, 1]   
EV Charger                                                                       [0, 1]   
instant_bookable                                                          [False, True]   
Elevator                                                                         [1, 0]   
BBQ Utensils                                                                     [0, 1]   
Baby and Children Equipments                                                     [0, 1]   
Bathtub                                                                          [0, 1]   
Beach Essentials                                                                 [0, 1]   
Breakfast                                                                        [0, 1]   
Building Staff                                                                   [0, 1]   
Air Conditioning                                                                 [1, 0]   
Cleaning Equipments                                                   

In [15]:
missing_numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values=np.nan))
])

In [16]:
missing_onehot_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent', missing_values=np.nan)),
    ('onehot', OneHotEncoder(drop='first'))
])

In [17]:
transformer = ColumnTransformer([
    ('missing_numerical', missing_numerical_pipeline, ['bedrooms', 'beds', 'total_bathrooms']),
    ('missing_onehot', missing_onehot_pipeline, ['bathrooms_type']), 
    ('nonmissing_onehot', OneHotEncoder(drop='first'), ['instant_bookable', 
                                                        'room_type', 'neighbourhood_group_cleansed']),
    ('nonmissing_binary', ce.BinaryEncoder(drop_invariant=True), ['neighbourhood_cleansed', 
                                                                  'property_type'])
], remainder='passthrough')

___
# Comparing Models
___

## CV Score From Train Dataset

In [81]:
model_result = pd.DataFrame()

In [82]:
for file in os.listdir('Models/Trained Models/'):
    if 'model_result' in file:
        with open('Models/Trained Models/'+file, 'rb') as f:
            result = pickle.load(f)
        model_result = model_result.append(result)

In [83]:
model_result = model_result.reset_index().rename(columns={0: 'MSE', 'index':'Model'})

In [84]:
model_result['RMSE'] = np.sqrt(model_result.MSE)

In [85]:
model_result.style.background_gradient(cmap='viridis_r', subset=['RMSE', 'MSE'])

In the table above, we can see that Tuned RFR models has the lowest RMSE of all models. Next, let's try to see its RMSE metric for train set and test set and determine which one is better

## Selecting Best Models

In [95]:
models = [('benchmark_dtr', 'benchmark_dtr.pkl'),
          ('tuned_dtr', 'tuned_dtr.pkl'),
          ('benchmark_gbr', 'benchmark_gbr.pkl'),
          ('tuned_gbr', 'tuned_gbr.pkl'),
          ('benchmark_rfr', 'benchmark_rfr.pkl'),
          ('tuned_rfr', 'tuned_rfr.pkl')]

In [110]:
final_result = pd.DataFrame()

In [111]:
for m in models:
    with open('Models/Trained Models/'+m[1], 'rb') as f:
        model = pickle.load(f)
        
    model.fit(X_train, y_train)
    
    predict_train = model.predict(X_train)
    mse_train = mean_squared_error(y_train, predict_train)
    rmse_train = np.sqrt(mse_train)
    
    predict_test = model.predict(X_test)
    mse_test = mean_squared_error(y_test, predict_test)
    rmse_test = np.sqrt(mse_test)
    
    result = {m[0] : {'MSE Train': mse_train, 'RMSE_Train': rmse_train, 
                      'MSE Test': mse_test, 'RMSE Test': rmse_test}}

    final_result = pd.concat([final_result, pd.DataFrame(result)], axis=1)

c:\users\zylveyn\appdata\local\programs\python\python39\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\users\zylveyn\appdata\local\programs\python\python39\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\users\zylveyn\appdata\local\programs\python\python39\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\users\zylveyn\appdata\local\programs\python\python39\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_cate

In [112]:
final_result.T.style.background_gradient('viridis')

From the table above, we can see the MSE and RMSE values for each model in train and test dataset after fitted with train dataset. We may notice that Decision Tree models (Benchmark and Tuned model) perform pretty amazing on train dataset but really worse on test dataset which may indicates overfitting.

By comparing all the models and its performance above, the best model would be Tuned Random Forest Regressor since it has relatively small value of MSE and RMSE in both train and test dataset.

## Saving and Exporting The Best Model

In [113]:
with open('Models/Trained Models/'+'tuned_rfr.pkl', 'rb') as f:
    best_model = pickle.load(f)

In [114]:
best_model.fit(X, y)

c:\users\zylveyn\appdata\local\programs\python\python39\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('missing_numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent'))]),
                                                  ['bedrooms', 'beds',
                                                   'total_bathrooms']),
                                                 ('missing_onehot',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='first'))]),
                                                  ['bathrooms_type']),
   

In [116]:
filename = 'best_model.pkl'
pickle.dump(best_model, open('Models/Trained Models/'+filename, 'wb'))